In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.227
[1,  4000] loss: 1.884
[1,  6000] loss: 1.718
[1,  8000] loss: 1.606
[1, 10000] loss: 1.532
[1, 12000] loss: 1.505
[2,  2000] loss: 1.432
[2,  4000] loss: 1.380
[2,  6000] loss: 1.367
[2,  8000] loss: 1.343
[2, 10000] loss: 1.304
[2, 12000] loss: 1.316
[3,  2000] loss: 1.256
[3,  4000] loss: 1.254
[3,  6000] loss: 1.225
[3,  8000] loss: 1.202
[3, 10000] loss: 1.212
[3, 12000] loss: 1.205
[4,  2000] loss: 1.140
[4,  4000] loss: 1.127
[4,  6000] loss: 1.144
[4,  8000] loss: 1.143
[4, 10000] loss: 1.131
[4, 12000] loss: 1.146
[5,  2000] loss: 1.057
[5,  4000] loss: 1.060
[5,  6000] loss: 1.064
[5,  8000] loss: 1.088
[5, 10000] loss: 1.068
[5, 12000] loss: 1.069
[6,  2000] loss: 1.006
[6,  4000] loss: 0.998
[6,  6000] loss: 1.014
[6,  8000] loss: 1.005
[6, 10000] loss: 1.029
[6, 12000] loss: 1.023
[7,  2000] loss: 0.938
[7,  4000] loss: 0.941
[7,  6000] loss: 0.957
[7,  8000] loss: 0.981
[7, 10000] loss: 0.986
[7, 12000] loss: 0.991
[8,  2000] loss: 0.900
[8,  4000] 

In [6]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 61 %
